In [137]:
import re

identifiers = '[_a-zA-Z][_a-zA-Z0-9]*'
string = '"[a-zA-Z][a-zA-Z0-9]*"'
ints = '[0-9][0-9]*'
floatsone = '\d+\.\d*'
floatstwo = '\.\d+'
octal = '[0][0-7]{1,2}'

In [138]:
# Available operations in the language
operations = {
'=':'ASSIGN_OPER', "!":"NEGATIVE",
'+':'ADDITION_OPER', '-':'SUB_OPER', 
'*':'MULT_OPER', '/':'DIV_OPER', 
'>':"GREATER_THAN", '<':"LESS_THAN",
'%':'MOD_OPER', '\'':'SINGLE_QUOTES', 
'"':'DOUBLE_QUOTES', '(':'LEFT_PARANTHESIS',
')':'RIGHT_PARANTHESIS', ":":"COLON",
'{':'LEFT_CURL','}':'RIGHT_CURL', 
';':'DELIMETER'
}

In [139]:
# These are the keywords of our language
keywords = {
'void':'VOID_CODE', 'main':'MAIN_CODE', 
'int':"INT_CODE", 'char':'CHAR_CODE',
'unsigned':'UNSIGNED_CODE', 'signed':'SIGNED_CODE',
'short':'SHORT_CODE', 'long':'LONG_CODE',
'float':'FLOAT_CODE', 'double':'DOUBLE_CODE', 
'struct':'STRUCT_CODE', 'default':'DEFAULT_CODE',
'break':"BREAK_CODE", 'continue':'CONTINUE_CODE',
'if':'IF_CODE','else':'ELSE_CODE',
'for':'FOR_CODE','foreach':'FOREACH_CODE',
'switch':'SWITCH_CODE', 'case':'CASE_CODE',
'enum':"ENUM_CODE", 'const':'CONST_CODE', 
'while':'WHILE_CODE', 'do':'DO_CODE',
'goto':'GOTO_CODE','sizeof':'SIZEOF_CODE',
'return':'RETURN_CODE','static':'STATIC_CODE'
}

In [140]:
# Just to store values in the process
value = ''

# This is an array for the parsed values
parses = []

with open('start.c','r') as file:

    while True:
        read_character = file.read(1)

        if read_character == "":
            break

        token = read_character
        identifier = re.fullmatch(identifiers, read_character)

        if identifier:
            val = 'IDENTIFIER'
            
        integers = re.fullmatch(ints, read_character)
        floats = re.fullmatch(floatsone, read_character)
        strings = re.fullmatch(string, read_character)

        if integers:
            val='INTEGER'
            
        if read_character == '"':
            next_character = ''
            while next_character != '"':
                next_character = file.read(1)
                temp = token + next_character
                token = temp
                if(next_character == '"'):
                    string_match = re.fullmatch(string, temp)
                    if string_match:
                        val = "STRING" 
                        
        if val == "STRING":
            if token != " ":
                parses.append((token,val))
                token=''
                val=""

        if read_character == ".":
            while read_character != '':
                next_character = file.read(1)
                if next_character == "":
                    break
                if next_character == " ":
                    break
                temp = token + next_character
                float_dot = re.fullmatch(floatstwo, temp)
                if float_dot:
                    token = temp
                    val = "FLOAT" 
                if next_character in operations.keys():
                    value = operations[next_character]  
                else:
                    break

        if read_character != "" and read_character in operations.keys():
            val = operations[read_character]
            parses.append((read_character, val))  

        while read_character != '' and identifier: 
            next_character = file.read(1)
            if next_character == "":
                break
            if next_character == " ":
                break
            temp = token + next_character
            identifier = re.fullmatch(identifiers, temp)
            if identifier:
                token = temp
                val="IDENTIFIER"
            if next_character in operations.keys():
                value = operations[next_character] 

        while read_character != '' and integers or floats:
            next_character = file.read(1)
            if next_character == "":
                break
            if next_character == " ":
                break
            temp = token + next_character
            integers = re.fullmatch(ints, temp)
            floats = re.fullmatch(floatsone, temp)
            if floats:
                token = temp
                val = "FLOAT"
            elif integers:
                token = temp
                val="INTEGER"
            if next_character in operations.keys():
                value = operations[next_character]
        
        if val == "IDENTIFIER" or val == "INTEGER" or val=="FLOAT" or val=="OCTAL"and token != " " and token != "\n":
            if token != " ":
                parses.append((token,val))
            token=''
            val=""
        if value and next_character != " " and next_character != "\n":
            parses.append((next_character, value))
            value=''
        token=''
        val=""
        if read_character== "":
            break 

In [141]:
for i in range(len(parses)):
    if parses[i][1] == 'IDENTIFIER':
        if parses[i][0] in keywords.keys():
            parse_temp = list([parses[i][0],keywords[parses[i][0]]])
            parses[i] = tuple(parse_temp)

i = 0
while i < len(parses):
    if parses[i][0] == "=" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = ("==", 'EQUALITY_OPERATOR')
    else:
        i +=1

i = 0
while i < len(parses):
    if parses[i][0] == ">" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = (">=", 'GREAT_EQUAL')
    else:
        i +=1

i = 0
while i < len(parses):
    if parses[i][0] == "<" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = ("<=", 'LESS_EQUAL')
    else:
        i +=1

i = 0
while i < len(parses):
    if parses[i][0] == "!" and parses[i+1][0] == "=":
        parses.pop(i+1)
        parses[i] = ("!=", 'NOT_EQUAL')
    else:
        i +=1

for i in parses:
    print(i)

('void', 'VOID_CODE')
('main', 'MAIN_CODE')
('(', 'LEFT_PARANTHESIS')
(')', 'RIGHT_PARANTHESIS')
('{', 'LEFT_CURL')
('int', 'INT_CODE')
('_test123', 'IDENTIFIER')
('=', 'ASSIGN_OPER')
('123', 'INTEGER')
(';', 'DELIMETER')
('return', 'RETURN_CODE')
('"hello"', 'STRING')
('"', 'DOUBLE_QUOTES')
(';', 'DELIMETER')
('}', 'RIGHT_CURL')


In [142]:
print(parses)

[('void', 'VOID_CODE'), ('main', 'MAIN_CODE'), ('(', 'LEFT_PARANTHESIS'), (')', 'RIGHT_PARANTHESIS'), ('{', 'LEFT_CURL'), ('int', 'INT_CODE'), ('_test123', 'IDENTIFIER'), ('=', 'ASSIGN_OPER'), ('123', 'INTEGER'), (';', 'DELIMETER'), ('return', 'RETURN_CODE'), ('"hello"', 'STRING'), ('"', 'DOUBLE_QUOTES'), (';', 'DELIMETER'), ('}', 'RIGHT_CURL')]


In [143]:
def error():
    print("Something went wrong")
    exit()

In [144]:
def lex():
    global nextToken
    if parses:
        val = parses.pop(0)
        val = val[1]
        nextToken = val
        print(nextToken)
    else:
        nextToken = None

def stmt():
    lex()
    if nextToken == 'INT_CODE' or nextToken == 'FLOAT_CODE' or nextToken == 'DOUBLE_CODE' or nextToken=='IDENTIFIER':
        assignment_stmt()
    elif nextToken == 'WHILE_CODE':
        while_stmt()
    elif nextToken == 'FOR_CODE':
        for_stmt()
    elif nextToken == 'IF_CODE':
        if_stmt()
    elif nextToken == 'RETURN_CODE':
        return_stmt()
    elif nextToken == 'FOREACH_CODE':
        foreach_stmt()
    elif nextToken == 'DO_CODE':
        do_while_stmt()
    elif nextToken == 'LEFT_CURL':
        block()
    elif nextToken == 'SWITCH_CODE':
        switch_stmt()

def block():
    print("STARTED <block>")
    if nextToken != 'LEFT_CURL':
        error()
    while nextToken != 'RIGHT_CURL' and parses:
        stmt()
    if nextToken == 'RIGHT_CURL':
        print("FINISHED <block>")
    else:
        error()

def assignment_stmt():
    print('STARTED <assign statement>')
    if nextToken == 'INT_CODE' or nextToken == 'FLOAT_CODE' or nextToken=='DOUBLE_CODE':
        lex()
        if nextToken != 'IDENTIFIER':
            error()
        lex()
        if nextToken == 'ASSIGN_OPER':
            lex()
            expr()
            if nextToken != 'DELIMETER':
                error()
            print('FINISHED <assignement statement>')
            return
        elif nextToken == "DELIMETER":
            print('FINISHED <assignment statement>')
        else:
            error()
    elif nextToken == 'IDENTIFIER':
        lex()
        if nextToken == 'ASSIGN_OPER':
            lex()
            expr()
            if nextToken != 'DELIMETER':
                error()
            print('FINISHED <assignement statement>')
        else:
            error()

    else:
        error()

def expr():
    print("STARTED <expr>")
    term()
    while (nextToken == 'ADDITION_OPER' or nextToken == 'SUB_OPER'):
        lex()
        term()
    print("FINISHED <expr>")
    
def term():
    print("STARTED <term>")
    factor()

    while(nextToken == 'MULT_OPER' or nextToken == 'DIV_OPER' or nextToken == 'MOD_OPER'):
        lex()
        factor()
    print("FINISHED <term>")
    
def factor():
    print("STARTED <factor>")
    if (nextToken == 'IDENTIFIER' or nextToken == 'INTEGER' or nextToken == 'FLOAT'):
        lex()

    elif nextToken == 'LEFT_PARANTHESIS':
        lex()
        expr()
        if nextToken == 'RIGHT_PARANTHESIS':
            lex()
        else:
            error()
    else:
        error()
    print("FINISHED <factor>")

def bool_expr():
    print("STARTED <bool_expr>")
    expr()
    if nextToken == "EQUALITY_OPERATOR" or nextToken == "NOT_EQUAL" or nextToken == "GREAT_EQUAL" or nextToken =="LESS_EQUAL" or nextToken == "GREATER_THAN" or nextToken == "LESS_THAN":
        lex()
        expr()
    else:
        error()
    print("FINISHED <bool_expr>")

def if_stmt():
    print("STARTED <IF STMT>")
    if (nextToken != 'IF_CODE'):
        error()
    else:
        lex()
        if (nextToken != 'LEFT_PARANTHESIS'):
            error()
        else:
            lex()
            bool_expr()
            if nextToken != 'RIGHT_PARANTHESIS':
                error()
            else:
                stmt()
                lex()
                if nextToken == 'ELSE_CODE':
                    print("STARTED <ELSE STMT>")
                    stmt()
                    print("FINISHED <ELSE STMT>")
    print("FINISHED <IF STMT>")

def for_stmt():
    print("STARTED <FOR STATMENT>")
    if nextToken != 'FOR_CODE':
        error()
    lex()
    if nextToken!='LEFT_PARANTHESIS':
        error()
    while nextToken != 'DELIMETER' and parses:
        lex()
    if nextToken != 'DELIMETER':
        error()
    lex()
    while nextToken != 'DELIMETER' and parses:
        lex()
    if nextToken != 'DELIMETER':
        error()
    while nextToken != 'RIGHT_PARANTHESIS' and parses:
        lex()
    if nextToken != 'RIGHT_PARANTHESIS':
        error()
    stmt()
    print("FINISHED <for statement>") 

def foreach_stmt():
    print("STARTED <foreach statement>")
    if nextToken != "FOREACH_CODE":
        error()
    lex()
    if nextToken != 'LEFT_PARANTHESIS':
        error()
    lex()
    if nextToken != 'IDENTIFIER':
        error()
    lex()
    if nextToken != 'COLON':
        error()
    lex()
    if nextToken != 'IDENTIFIER':
        error()
    lex()
    if nextToken != 'RIGHT_PARANTHESIS':
        error()
    else:
        stmt()
    print('FINISHED <foreach statement>')

def while_stmt():
    print('STARTED <while statement>')
    if nextToken != 'WHILE_CODE':
        error()
    else:
        lex()
        if nextToken != 'LEFT_PARANTHESIS':
            error()
        else:
            lex()
            bool_expr()
            if nextToken != 'RIGHT_PARANTHESIS':
                error()
            else:
                stmt()
    print("FINISHED <while statement>")

def switch_stmt():
    print('STARTED <switch statement>')
    if nextToken != 'SWITCH_CODE':
        error()
    else:
        lex()
        if nextToken != 'LEFT_PARANTHESIS':
            error()
        lex()
        expr()
        if nextToken != 'RIGHT_PARANTHESIS':
            error()
        lex()
        if nextToken != 'LEFT_CURL':
            error()
        
        lex()
        if nextToken == 'CASE_CODE' or nextToken == 'DEFAULT_CODE':
            lex()
            expr()
            if nextToken != 'COLON':
                error()
            stmt()
        else:
            if nextToken != 'RIGHT_CURL':
                error()
        print('FINISHED <switch statement>')

def return_stmt():
    print("STARTED <Return statement>")
    if nextToken != 'RETURN_CODE':
        error()
    lex()
    expr()
    print('FINISHED <return statement>')

def do_while_stmt():
    print('STARTED <Do While Statement>')
    if nextToken != 'DO_CODE':
        error()
    stmt()
    lex()
    if nextToken != "WHILE_CODE":
        error()
    lex()
    if nextToken != "LEFT_PARANTHESIS":
        error()
    lex()
    bool_expr()
    if nextToken != 'RIGHT_PARANTHESIS':
        print(nextToken)
        error()
    lex()
    if nextToken != 'DELIMETER':
        error()
    print("FINISHED <Do While Statement>")

In [145]:
def program():
    print('STARTED <Program>')
    lex()
    if nextToken != 'VOID_CODE':
        error()
    lex()
    if nextToken != 'MAIN_CODE':
        error()
    lex()
    if nextToken != 'LEFT_PARANTHESIS':
        error()
    lex()
    if nextToken != 'RIGHT_PARANTHESIS':
        error()
    else:
        stmt()
    print('FINISHED <Program>')                 

program()

STARTED <Program>
VOID_CODE
MAIN_CODE
LEFT_PARANTHESIS
RIGHT_PARANTHESIS
LEFT_CURL
STARTED <block>
INT_CODE
STARTED <assign statement>
IDENTIFIER
ASSIGN_OPER
INTEGER
STARTED <expr>
STARTED <term>
STARTED <factor>
DELIMETER
FINISHED <factor>
FINISHED <term>
FINISHED <expr>
FINISHED <assignement statement>
RETURN_CODE
STARTED <Return statement>
STRING
STARTED <expr>
STARTED <term>
STARTED <factor>
Something went wrong
FINISHED <factor>
FINISHED <term>
FINISHED <expr>
FINISHED <return statement>
DOUBLE_QUOTES
DELIMETER
RIGHT_CURL
FINISHED <block>
FINISHED <Program>
